In [10]:
import uuid
from googleads import adwords
import os
import datetime
import pandas as pd
auth_file_path = '/home/jian/Docs/Google_Ads_Access/googleads_BL_MMC.yaml'

def upload_and_return_count_in_chunk(service, user_list_id, email_list):
    ids_in_chunk=0
    mutate_members_operation = {
      'operand': {
          'userListId': user_list_id,
          'membersList': email_list
      },
      'operator': 'ADD'
    }
    response = service.mutateMembers([mutate_members_operation])
    if 'userLists' in response:
        for user_list in response['userLists']:
            # print('Uploaded %i items to user list with name "%s" and ID "%d"'
            #     % (len(email_list), user_list['name'], user_list['id']))

            
            ids_in_chunk+=len(email_list)
    return ids_in_chunk
            
def run_the_uploading(client, email_file_path):
  # Initialize appropriate services.
    total_ids_in_seg=0
    user_list_service = client.GetService('AdwordsUserListService', 'v201809')
    file_base_name=os.path.basename(email_file_path).split(".")[0]
    user_list = {
        'xsi_type': 'CrmBasedUserList',
        'name': file_base_name+"_test_no_use2",
        'description': 'A list of customers that originated from email addresses as '+file_base_name,
        # CRM-based user lists can use a membershipLifeSpan of 10000 to indicate
        # unlimited; otherwise normal values apply.
        'membershipLifeSpan': 10000,
        'uploadKeyType': 'CONTACT_INFO'
     }
    # Create an operation to add the user list.
    operations = [{
        'operator': 'ADD',
        'operand': user_list
    }]
    result = user_list_service.mutate(operations)
    user_list_id = result['value'][0]['id']

    df = pd.read_csv(email_file_path, chunksize=3000)
    for i in df:
        mail_col = [k for k in i.columns if 'email' in k.lower()][0]
        new_col = ['email' if h == mail_col else h for h in i.columns.tolist()]
        i.columns = new_col
        emails = i['email']
        emails = set([i for i in emails if str(i) != 'nan'])
        members = [{'hashedEmail': email} for email in emails]
        ids_in_chunk=upload_and_return_count_in_chunk(user_list_service, user_list_id, members)
        
        total_ids_in_seg+=ids_in_chunk
    print(datetime.datetime.now(),"all %s ids uploaded in the segment "%str(total_ids_in_seg))

In [11]:
writer_floder='/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Google_uploading_audience_JT_20200129/audience_files/'

import glob

list_files_to_upload=glob.glob(writer_floder+"*.csv")

print(len(list_files_to_upload))
list_files_to_upload

4


['/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Google_uploading_audience_JT_20200129/audience_files/BL_2020Q1_Exp_income_50_above.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Google_uploading_audience_JT_20200129/audience_files/BL_2020Q1_Exp_income_50_below.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Google_uploading_audience_JT_20200129/audience_files/BL_2020Q1_Exp_LiveDesc_SingleFamily.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Google_uploading_audience_JT_20200129/audience_files/BL_2020Q1_Exp_LiveDesc_MultipleFamily.csv']

In [ ]:
BL_client = adwords.AdWordsClient.LoadFromStorage(auth_file_path)
for local_seg_file in list_files_to_upload[:1]:
    run_the_uploading(BL_client, local_seg_file)